In [54]:
!pip install synthcity deap
!pip install opacus==1.4.0


In [55]:
# !unrar x data.rar

In [56]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from hyperopt import space_eval
from synthcity.plugins import Plugins
from synthcity.plugins.generic.plugin_ddpm import TabDDPMPlugin as DDPM
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor

In [57]:
dir_datasets = '/kaggle/input/nir02-main/data/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

# Словарь датасетов для удобства
datasets = {
        'insurance': {
                        "data": real_data_1,
                        "task": "regression",
                        "target": "expenses",
                        "num_columns":
                        ["age", "bmi", "children"],
                        "cat_columns":
                        ["sex", "smoker", "region"]
                    },
        'my_classification': {
                        "data": real_data_2,
                        "task": "classification",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3", "feature_4"],
                        "cat_columns":
                        []
                    },
        'my_regression': {
                        "data": real_data_3,
                        "task": "regression",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3"],
                        "cat_columns":
                        []
                    },

        'two_moons': {
                        "data": real_data_4,
                        "task": "classification",
                        "target": "label",
                        "num_columns":
                        ["x1", "x2"],
                        "cat_columns": 
                        []
                    },
        'california_housing': {
                        "data": real_data_5,
                        "task": "regression",
                        "target": "MedHouseVal",
                        "num_columns":
                        ["MedInc",	"HouseAge",	"AveRooms",	"AveBedrms",	"Population",	"AveOccup",	"Latitude",	"Longitude"],
                        "cat_columns": 
                        []
                    },
        'titanic': {
                        "data": real_data_6,
                        "task": "classification",
                        "target": "Survived",
                        "num_columns": ["Age", "Fare", "SibSp", "Parch"],
                        "cat_columns": ["Pclass", "Sex", "Embarked"]
                    },

}

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете insurance:
 Количество строк: 1338
 Количество колонок: 7
 Колонки: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses']
 Задача: regression
 Целевая переменная: expenses
 Числовые признаки: ['age', 'bmi', 'children']
 Категориальные признаки: ['sex', 'smoker', 'region']

 Информация о датасете my_classification:
 Количество строк: 900
 Количество колонок: 6
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'target']
 Задача: classification
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
 Категориальные признаки: []

 Информация о датасете my_regression:
 Количество строк: 900
 Количество колонок: 5
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'target']
 Задача: regression
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3']
 Категориальные признаки: []

 Информация о датасете two_moons:
 Количе

In [58]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [59]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [60]:
# DDPM пространство
ddpm_space = {
    'batch_size': [4096],  # дискретный
    'lr': (3.5e-4, 9.2e-4, 1e-5),  # непрерывный с квантованием
    'num_timesteps': [1000],  # дискретный
    'n_layers_hidden': [2, 4, 6],  # дискретный
    'n_units_hidden': [256, 512, 1024],  # дискретный
    'transformation_num_type': ['None'],  # дискретный
    'transformation_cat_type': ['None']  # дискретный
}

In [61]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [62]:
import numpy as np
import random
from deap import base, creator, tools, algorithms
from hyperopt import hp
from sklearn.model_selection import KFold
import warnings
from contextlib import contextmanager
import sys
import os

@contextmanager
def suppress_all_output():
    """Контекстный менеджер для подавления всего вывода"""
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

class UniversalGeneticOptimizer:
    def __init__(self,
                 param_space,
                 population_size=20,
                 n_generations=10,
                 mutation_prob=0.2,
                 crossover_prob=0.8,
                 tournament_size=3,
                 random_seed=42,
                 verbose=True):
        """
        Универсальный генетический алгоритм для оптимизации гиперпараметров

        Args:
            param_space: словарь с определением пространства поиска
            population_size: размер популяции
            n_generations: количество поколений
            mutation_prob: вероятность мутации
            crossover_prob: вероятность скрещивания
            tournament_size: размер турнира для отбора
            random_seed: seed для воспроизводимости
            verbose: вывод прогресса
        """
        self.param_space = param_space
        self.population_size = population_size
        self.n_generations = n_generations
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        self.tournament_size = tournament_size
        self.verbose = verbose

        # Устанавливаем seed
        random.seed(random_seed)
        np.random.seed(random_seed)

        # Анализируем пространство параметров
        self.param_info = self._analyze_param_space()

        # Настройка DEAP
        self._setup_deap()

        # История оптимизации
        self.history = {
            'generations': [],
            'best_fitness': [],
            'avg_fitness': [],
            'best_params': []
        }

    def _analyze_param_space(self):
        """Анализ пространства параметров для определения их типов"""
        param_info = {}

        for param_name, param_def in self.param_space.items():
            if isinstance(param_def, tuple) and len(param_def) == 2:
                # Непрерывный параметр (min, max)
                param_info[param_name] = {
                    'type': 'continuous',
                    'min': param_def[0],
                    'max': param_def[1],
                    'quantization': None
                }
            elif isinstance(param_def, tuple) and len(param_def) == 3:
                # Непрерывный параметр с квантованием (min, max, step)
                param_info[param_name] = {
                    'type': 'continuous',
                    'min': param_def[0],
                    'max': param_def[1],
                    'quantization': param_def[2]
                }
            elif isinstance(param_def, list):
                # Дискретный параметр
                param_info[param_name] = {
                    'type': 'discrete',
                    'values': param_def
                }
            else:
                raise ValueError(f"Неподдерживаемый тип параметра: {param_name}")

        return param_info

    def _setup_deap(self):
        """Настройка DEAP для минимизации"""
        # Создаем класс для фитнеса (минимизация)
        if not hasattr(creator, "FitnessMin"):
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        if not hasattr(creator, "Individual"):
            creator.create("Individual", list, fitness=creator.FitnessMin)

        self.toolbox = base.Toolbox()

        # Регистрируем функции создания индивидов
        self.toolbox.register("individual", self._create_individual)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)

        # Регистрируем операторы
        self.toolbox.register("mate", self._crossover)
        self.toolbox.register("mutate", self._mutate)
        self.toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)
        self.toolbox.register("evaluate", self._evaluate_individual)

    def _create_individual(self):
        """Создание случайного индивида"""
        individual = []

        for param_name in self.param_space.keys():
            info = self.param_info[param_name]

            if info['type'] == 'continuous':
                # Непрерывный параметр
                value = np.exp(np.random.uniform(np.log(info['min']), np.log(info['max'])))
                if info['quantization'] is not None:
                    # Применяем квантование
                    value = round(value / info['quantization']) * info['quantization']
                    value = np.clip(value, info['min'], info['max'])
                individual.append(value)

            elif info['type'] == 'discrete':
                # Дискретный параметр - сохраняем индекс
                individual.append(np.random.randint(len(info['values'])))

        return creator.Individual(individual)

    def _decode_individual(self, individual):
        """Декодирование индивида в параметры"""
        params = {}

        for i, param_name in enumerate(self.param_space.keys()):
            info = self.param_info[param_name]

            if info['type'] == 'continuous':
                params[param_name] = individual[i]
            elif info['type'] == 'discrete':
                params[param_name] = info['values'][int(individual[i])]

        return params

    def _crossover(self, ind1, ind2):
        """One-point crossover"""
        # Выбираем случайную точку разреза
        if len(ind1) > 1:
            crossover_point = random.randint(1, len(ind1) - 1)
        else:
            crossover_point = 1

        # Обмениваемся частями после точки разреза
        for i in range(crossover_point, len(ind1)):
            ind1[i], ind2[i] = ind2[i], ind1[i]

        return ind1, ind2

    def _mutate(self, individual):
        """Операция мутации - случайное значение из пространства поиска"""
        for i, param_name in enumerate(self.param_space.keys()):
            if random.random() < self.mutation_prob:
                info = self.param_info[param_name]

                if info['type'] == 'continuous':
                    # Генерируем новое случайное значение
                    new_value = np.exp(np.random.uniform(np.log(info['min']), np.log(info['max'])))
                    if info['quantization'] is not None:
                        new_value = round(new_value / info['quantization']) * info['quantization']
                        new_value = np.clip(new_value, info['min'], info['max'])
                    individual[i] = new_value

                elif info['type'] == 'discrete':
                    # Выбираем случайный индекс
                    individual[i] = np.random.randint(len(info['values']))

        return individual,

    def _evaluate_individual(self, individual):
        """Оценка индивида"""

        params = self._decode_individual(individual)
        score = self.objective_function(params)

        return score,

    def set_objective_function(self, objective_func):
        """Установка целевой функции"""
        self.objective_function = objective_func

    def optimize(self):
        """Запуск оптимизации"""
        if self.verbose:
            print(f"Пространство параметров:")
            for param_name, info in self.param_info.items():
                if info['type'] == 'continuous':
                    quant_info = f", квантование: {info['quantization']}" if info['quantization'] else ""
                    print(f"  {param_name}: непрерывный ({info['min']}, {info['max']}){quant_info}")
                else:
                    print(f"  {param_name}: дискретный {info['values']}")
            print(f"Размер популяции: {self.population_size}")
            print(f"Количество поколений: {self.n_generations}")
            print(f"Вероятность мутации: {self.mutation_prob}")
            print(f"Вероятность скрещивания: {self.crossover_prob}")
            print("="*60)

        # Создание начальной популяции
        population = self.toolbox.population(n=self.population_size)

        # Оценка начальной популяции
        fitnesses = list(map(self.toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        if self.verbose:
            self._print_population_info(population, 0, "Начальная популяция")

        # Эволюция
        for generation in range(1, self.n_generations + 1):
            if self.verbose:
                print(f"\n{'='*20} ПОКОЛЕНИЕ {generation} {'='*20}")

            # Отбор родителей
            parents = self.toolbox.select(population, len(population))


            # Создание потомков
            offspring = list(map(self.toolbox.clone, parents))

            # Скрещивание (one-point crossover)
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.crossover_prob:
                    self.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

            # Мутация
            for mutant in offspring:
                if random.random() < self.mutation_prob:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Оценка потомков
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

            # ЭЛИТАРНАЯ СТРАТЕГИЯ: объединение родителей и потомков
            combined_population = population + offspring
            population = tools.selBest(combined_population, self.population_size)

            if self.verbose:
                sorted_combined = sorted(combined_population, key=lambda x: x.fitness.values[0])
                params = self._decode_individual(sorted_combined[0])

                print('Лучший индивид')
                print(f"ML-EFFICACY={-sorted_combined[0].fitness.values[0]}, Params={self._format_params(params)}")

            # Сохранение статистики
            self._save_generation_stats(population, generation)


        # Лучший индивид
        best_individual = tools.selBest(population, 1)[0]
        best_params = self._decode_individual(best_individual)
        best_fitness = best_individual.fitness.values[0]

        if self.verbose:
            print(f"\n{'='*60}")
            print("РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ:")
            print(f"Лучший ML-EFFICACY: {-best_fitness}")
            print(f"Лучшие параметры: {best_params}")
            print(f"{'='*60}")


        return {
            'best_params': best_params,
            'best_ml_eff': best_fitness,
            'ml_eff_diff_from_optimal': best_fitness,
            'history': self.history
        }

    def _format_params(self, params):
        """Форматирование параметров для вывода"""
        formatted = {}
        for key, value in params.items():
            if isinstance(value, float):
                formatted[key] = f"{value}"
            else:
                formatted[key] = value
        return formatted

    def _print_population_info(self, population, generation, title):
        """Вывод информации о популяции"""
        fitnesses = [ind.fitness.values[0] for ind in population]
        best_fitness = min(fitnesses)
        avg_fitness = np.mean(fitnesses)

        print(f"\n{title}:")
        print(f"  Лучший ML-EFFICACY: {-best_fitness}")
        print(f"  Средний ML-EFFICACY: {-avg_fitness}")


    def _save_generation_stats(self, population, generation):
        """Сохранение статистики поколения"""
        fitnesses = [ind.fitness.values[0] for ind in population]
        best_fitness = min(fitnesses)
        avg_fitness = np.mean(fitnesses)

        best_individual = min(population, key=lambda x: x.fitness.values[0])
        best_params = self._decode_individual(best_individual)

        self.history['generations'].append(generation)
        self.history['best_fitness'].append(best_fitness)
        self.history['avg_fitness'].append(avg_fitness)
        self.history['best_params'].append(best_params)


In [63]:
def evaluate_ml_efficacy(real_train, synthetic_train, real_test, target_column, num_columns, cat_columns, task_type='classification'):
    """
    Вычисляет ML-Efficacy метрику

    Args:
        real_train: реальные тренировочные данные
        synthetic_train: синтетические тренировочные данные
        real_test: реальные тестовые данные
        target_column: название целевой колонки
        task_type: 'classification' или 'regression'

    Returns:
        ml_efficacy_score: отношение качества модели на синтетических данных к качеству на реальных
    """

    # Подготовка данных
    real_train_copy = real_train.copy()

    # Обработка синтетических данных (если это объект с методом dataframe)
    if hasattr(synthetic_train, "dataframe"):
        synthetic_train = synthetic_train.dataframe()
    synthetic_train_copy = synthetic_train.copy()

    real_test_copy = real_test.copy()

    # Определение категориальных колонок (исключая целевую)
    feature_columns = num_columns
    categorical_columns = cat_columns

    # Label Encoding для категориальных признаков
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        # Обучаем на реальных данных
        combined_values = pd.concat([real_train_copy[col], real_test_copy[col]]).astype(str)
        le.fit(combined_values)
        label_encoders[col] = le

        # Применяем к реальным данным
        real_train_copy[col] = le.transform(real_train_copy[col].astype(str))
        real_test_copy[col] = le.transform(real_test_copy[col].astype(str))

        # Применяем к синтетическим данным, обрабатываем неизвестные значения
        synthetic_values = synthetic_train_copy[col].astype(str)
        known_values = set(le.classes_)
        synthetic_values = synthetic_values.apply(lambda x: x if x in known_values else le.classes_[0])
        synthetic_train_copy[col] = le.transform(synthetic_values)

    # Разделение на признаки и целевую переменную
    X_real_train = real_train_copy.drop(columns=[target_column])
    y_real_train = real_train_copy[target_column]

    X_synthetic_train = synthetic_train_copy.drop(columns=[target_column])
    y_synthetic_train = synthetic_train_copy[target_column]

    X_real_test = real_test_copy.drop(columns=[target_column])
    y_real_test = real_test_copy[target_column]

    # Выбор модели в зависимости от типа задачи
    if task_type == 'classification':
        model_real = XGBClassifier(random_state=42, eval_metric='logloss')
        model_synthetic = XGBClassifier(random_state=42, eval_metric='logloss')
        score_func = f1_score
        score_kwargs = {'average': 'weighted'} if len(np.unique(y_real_train)) > 2 else {'average': 'binary'}
    else:  # regression
        model_real = XGBRegressor(random_state=42)
        model_synthetic = XGBRegressor(random_state=42)
        score_func = r2_score
        score_kwargs = {}

    try:
        # Обучение модели на реальных данных
        model_real.fit(X_real_train, y_real_train)
        y_pred_real = model_real.predict(X_real_test)
        score_real = score_func(y_real_test, y_pred_real, **score_kwargs)

        # Обучение модели на синтетических данных
        model_synthetic.fit(X_synthetic_train, y_synthetic_train)
        y_pred_synthetic = model_synthetic.predict(X_real_test)
        score_synthetic = score_func(y_real_test, y_pred_synthetic, **score_kwargs)

        # Вычисление ML-Efficacy
        if score_real > 0:
            ml_efficacy = score_synthetic / score_real
        else:
            ml_efficacy = 0.0

        return ml_efficacy

    except Exception as e:
        print(f"Ошибка при вычислении ML-Efficacy: {e}")
        return 0.0

In [64]:
from sklearn.model_selection import KFold

def generate_and_evaluate_ml_efficacy(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # Извлекаем информацию о задаче и целевой переменной из dataset_info
    task_type = dataset_info['task']
    target_column = dataset_info['target']

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        params['transformation_num_type'], params['transformation_cat_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    ml_efficacy_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        # Создание и обучение DDPM с заданными параметрами
        ddpm = DDPM(
            lr=params['lr'],
            num_timesteps=params['num_timesteps'],
            model_params = dict(n_layers_hidden=params['n_layers_hidden'],
                                n_units_hidden=params['n_units_hidden'],
                                dropout=0.0),
            batch_size=params['batch_size']
        )

        # Обучение модели на реальных данных
        with suppress_all_output():
            ddpm.fit(train_data)

        # Генерация синтетических данных
        synthetic_data = ddpm.generate(len(train_data))

        cat_columns = dataset_info["cat_columns"].copy()
        if dataset_info["task"] == 'classification':
            cat_columns.append(dataset_info['target'])

        # Вычисляем ML-Efficacy
        score = evaluate_ml_efficacy(
            real_train=train_data,
            synthetic_train=synthetic_data,
            real_test=test_data,
            target_column=target_column,
            num_columns = dataset_info["num_columns"],
            cat_columns = cat_columns,
            task_type=task_type
        )
        ml_efficacy_scores.append(score)

    return -np.mean(ml_efficacy_scores) #Возращаем *(-1) для минимизации

In [65]:
def optimize_dataset_genetic(dataset_name, dataset_info, param_space,
                           population_size=20, n_generations=10,
                           mutation_prob=0.2, crossover_prob=1.0,
                           tournament_size=3, random_seed=None, verbose=True):
    """
    Универсальная оптимизация датасета с использованием генетического алгоритма

    Args:
        dataset_name: название датасета
        dataset: информация о датасете
        param_space: пространство поиска параметров
        population_size: размер популяции
        n_generations: количество поколений
        mutation_prob: вероятность мутации
        crossover_prob: вероятность скрещивания
        tournament_size: размер турнира
        random_seed: seed для воспроизводимости
        verbose: вывод прогресса
    """
    print(f"Универсальная генетическая оптимизация для датасета: {dataset_name}")

    # Создание оптимизатора
    optimizer = UniversalGeneticOptimizer(
        param_space=param_space,
        population_size=population_size,
        n_generations=n_generations,
        mutation_prob=mutation_prob,
        crossover_prob=crossover_prob,
        tournament_size=tournament_size,
        random_seed=random_seed,
        verbose=verbose
    )

    # Определение целевой функции
    def objective(params):
        if verbose:
            print(f"Оценка параметров: {optimizer._format_params(params)}")

        ml_eff_score = generate_and_evaluate_ml_efficacy(params, dataset_info)

        if verbose:
            print(f"ML-EFFICACY score: {-ml_eff_score}")

        return ml_eff_score

    optimizer.set_objective_function(objective)

    # Запуск оптимизации
    start_time = time.time()
    results = optimizer.optimize()
    end_time = time.time()  # время окончания

    elapsed_time = end_time - start_time
    formatted_time = str(timedelta(seconds=int(elapsed_time)))

    print(f"Лучший ML-EFFICACY для {dataset_name}: {-results['best_ml_eff']}")
    print(f"Лучшие параметры: {results['best_params']}")
    print(f"[INFO] Время работы алгоритма: {formatted_time}")
    print('-' * 50)

    return results

In [66]:
results = {}
N_GENERATIONS = 4 # без учета начальной популяции
POPULATION_SIZE = 10
MUTATION_PROB = 0.4
TOURNAMENT_SIZE = 3


for dataset_name in datasets:
    data = datasets[dataset_name]
    results[dataset_name] = optimize_dataset_genetic(
        dataset_name=dataset_name,
        dataset_info=datasets[dataset_name],
        param_space=ddpm_space,  # или ctgan_space, или любое другое
        population_size=POPULATION_SIZE,
        n_generations=N_GENERATIONS,
        mutation_prob=MUTATION_PROB,
        tournament_size=TOURNAMENT_SIZE,
        random_seed=RANDOM_SEED,
        verbose=True,
    )

Универсальная генетическая оптимизация для датасета: insurance
Пространство параметров:
  batch_size: дискретный [4096]
  lr: непрерывный (0.00035, 0.00092), квантование: 1e-05
  num_timesteps: дискретный [1000]
  n_layers_hidden: дискретный [2, 4, 6]
  n_units_hidden: дискретный [256, 512, 1024]
  transformation_num_type: дискретный ['None']
  transformation_cat_type: дискретный ['None']
Размер популяции: 10
Количество поколений: 4
Вероятность мутации: 0.4
Вероятность скрещивания: 1.0
Оценка параметров: {'batch_size': 4096, 'lr': '0.0005', 'num_timesteps': 1000, 'n_layers_hidden': 2, 'n_units_hidden': 1024, 'transformation_num_type': 'None', 'transformation_cat_type': 'None'}
ML-EFFICACY score: -2.144074565756142
Оценка параметров: {'batch_size': 4096, 'lr': '0.00071', 'num_timesteps': 1000, 'n_layers_hidden': 2, 'n_units_hidden': 256, 'transformation_num_type': 'None', 'transformation_cat_type': 'None'}
ML-EFFICACY score: 0.4160575106141344
Оценка параметров: {'batch_size': 4096, 'lr

In [67]:
for dataset_name in datasets:
    print(f"{dataset_name}: {-results[dataset_name]['best_ml_eff']}")

insurance: 0.8425765686574856
my_classification: 0.998781013169831
my_regression: 0.9986219588595714
two_moons: 0.9844246119788584
california_housing: 0.9684611063511506
titanic: 0.9817507783963073
